In [7]:
import sys
# Force a clean reinstall if the error persists
%pip uninstall ultralytics -y
%pip install ultralytics
# Verify the installation path and version
import ultralytics
print(f"Version: {ultralytics.__version__}")
print(f"Path: {ultralytics.__file__}")

Found existing installation: ultralytics 8.3.240
Uninstalling ultralytics-8.3.240:
  Successfully uninstalled ultralytics-8.3.240
Note: you may need to restart the kernel to use updated packages.
  Using cached ultralytics-8.3.240-py3-none-any.whl.metadata (37 kB)
Using cached ultralytics-8.3.240-py3-none-any.whl (1.1 MB)
Note: you may need to restart the kernel to use updated packages.
Version: 8.3.240
Path: C:\Users\yuval\AppData\Local\Programs\Python\Python312\Lib\site-packages\ultralytics\__init__.py


In [9]:
from ultralytics import YOLO
import os
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import random
from pathlib import Path
from PIL import Image


ImportError: cannot import name 'YOLO' from 'ultralytics' (C:\Users\yuval\AppData\Local\Programs\Python\Python312\Lib\site-packages\ultralytics\__init__.py)

In [16]:
dataset_path = r"C:\Users\yuval\PycharmProjects\GenAI_Project\dirty_imgs"
mud_images_path = os.path.join(dataset_path, r"mud\image_dirty")
mud_labels_path = os.path.join(dataset_path, r"mud\label_2")
water_images_path = os.path.join(dataset_path, r"water\image_dirty")
water_labels_path = os.path.join(dataset_path, r"water\label_2")

# Output directory for YOLO
yolo_output = r"C:\Users\yuval\PycharmProjects\GenAI_Project\yolo_dataset"
os.makedirs(os.path.join(yolo_output, "images/train"), exist_ok=True)
os.makedirs(os.path.join(yolo_output, "labels/train"), exist_ok=True)


In [ ]:
# --- Method 1: Label Conversion Helper ---
def convert_kitti_to_yolo(kitti_path, img_w, img_h):
    """Converts a single KITTI text file to YOLO normalized format."""
    if not os.path.exists(kitti_path): return ""
    yolo_lines = []
    with open(kitti_path, 'r') as f:
        for line in f.readlines():
            parts = line.split()
            if len(parts) < 15: continue
            # KITTI bbox: left, top, right, bottom (pixels)
            l, t, r, b = map(float, parts[4:8])
            # YOLO format: x_center, y_center, width, height (normalized 0-1)
            xc, yc = (l + r) / 2 / img_w, (t + b) / 2 / img_h
            w, h = (r - l) / img_w, (b - t) / img_h
            yolo_lines.append(f"0 {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}")
    return "\n".join(yolo_lines)

# --- Method 2: Dataset Preparation & YAML Integration ---
def prepare_dirty_dataset(dataset_root):
    """Processes labels, shuffles image paths, and generates the YAML config."""
    categories = ['mud', 'water']
    all_image_paths = []

    for cat in categories:
        img_dir = os.path.join(dataset_root, cat, "image_dirty")
        kitti_dir = os.path.join(dataset_root, cat, "label_2")
        yolo_lbl_dir = os.path.join(dataset_root, cat, "labels")
        os.makedirs(yolo_lbl_dir, exist_ok=True)

        print(f"Processing {cat} labels...")
        for img_name in os.listdir(img_dir):
            if img_name.endswith(".png"):
                img_path = os.path.abspath(os.path.join(img_dir, img_name))
                all_image_paths.append(img_path)

                # Convert label format
                kitti_file = os.path.join(kitti_dir, img_name.replace(".png", ".txt"))
                yolo_file = os.path.join(yolo_lbl_dir, img_name.replace(".png", ".txt"))

                with Image.open(img_path) as img:
                    w, h = img.size

                yolo_content = convert_kitti_to_yolo(kitti_file, w, h)
                with open(yolo_file, "w") as f:
                    f.write(yolo_content)

    # Shuffle to mix mud and water images
    random.seed(42)
    random.shuffle(all_image_paths)

    # Save the list of image paths
    list_path = os.path.join(dataset_root, "train_list.txt")
    with open(list_path, "w") as f:
        f.write("\n".join(all_image_paths))

    # Integrate YAML Generation
    yaml_path = os.path.join(dataset_root, "dataset.yaml")
    yaml_content = f"""
train: {list_path}
val: {list_path}
nc: 1
names: ['object']
"""
    with open(yaml_path, "w") as f:
        f.write(yaml_content)

    print(f"Dataset ready. YAML created at: {yaml_path}")
    return yaml_path

In [17]:
model = YOLO("yolov8s.pt")  # baseline pretrained model
yaml_config = prepare_dirty_dataset(dataset_path)

In [18]:
results = model.train(
    data=yaml_config,
    epochs=25,
    imgsz=640,
    batch=8,
    name="yolo8_dirty_imgs_shuffled_run"
)

Ultralytics 8.3.240  Python-3.12.9 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=kitti_dirty.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo8_dirty_run4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.

C:\Users\yuval\AppData\Local\Programs\Python\Python312\Lib\site-packages\ultralytics\utils\metrics.py:832: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
C:\Users\yuval\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\_core\_methods.py:137: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/20      2.18G          0      368.1          0          0        640: 100% ━━━━━━━━━━━━ 13/13 2.9it/s 4.5s0.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 6.9it/s 1.0s0.2s
                   all        100          0          0          0          0          0
WARNING no labels found in detect set, cannot compute metrics without labels


C:\Users\yuval\AppData\Local\Programs\Python\Python312\Lib\site-packages\ultralytics\utils\metrics.py:832: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
C:\Users\yuval\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\_core\_methods.py:137: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       3/20      2.21G          0      151.6          0          0        640: 100% ━━━━━━━━━━━━ 13/13 3.1it/s 4.2s0.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 6.3it/s 1.1s0.2s
                   all        100          0          0          0          0          0
WARNING no labels found in detect set, cannot compute metrics without labels


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      20/20      2.42G          0     0.2106          0          0        640: 100% ━━━━━━━━━━━━ 13/13 2.8it/s 4.6s0.3s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 11.6it/s 0.6s.1s
                   all        100          0          0          0          0          0
WARNING no labels found in detect set, cannot compute metrics without labels


C:\Users\yuval\AppData\Local\Programs\Python\Python312\Lib\site-packages\ultralytics\utils\metrics.py:832: RuntimeWarning: Mean of empty slice.
  i = smooth(f1_curve.mean(0), 0.1).argmax()  # max F1 index
C:\Users\yuval\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\_core\_methods.py:137: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(



20 epochs completed in 0.036 hours.
Optimizer stripped from C:\Users\yuval\PycharmProjects\GenAI_Project\runs\detect\yolo8_dirty_run4\weights\last.pt, 22.5MB
Optimizer stripped from C:\Users\yuval\PycharmProjects\GenAI_Project\runs\detect\yolo8_dirty_run4\weights\best.pt, 22.5MB

Validating C:\Users\yuval\PycharmProjects\GenAI_Project\runs\detect\yolo8_dirty_run4\weights\best.pt...
Ultralytics 8.3.240  Python-3.12.9 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 11.2it/s 0.6s.3s


C:\Users\yuval\AppData\Local\Programs\Python\Python312\Lib\site-packages\ultralytics\utils\metrics.py:653: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
C:\Users\yuval\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\_core\_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\yuval\AppData\Local\Programs\Python\Python312\Lib\site-packages\ultralytics\utils\metrics.py:697: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
C:\Users\yuval\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\_core\_methods.py:137: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
C:\Users\yuval\AppData\Local\Programs\Python\Python312\Lib\site-packages\ultralytics\utils\metrics.py:697: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
C:\Users\yuval\AppData\Loca

                   all        100          0          0          0          0          0
WARNING no labels found in detect set, cannot compute metrics without labels
Speed: 0.2ms preprocess, 5.2ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to C:\Users\yuval\PycharmProjects\GenAI_Project\runs\detect\yolo8_dirty_run4


In [19]:
results = model.predict(
        source=images_to_predict,
        conf=0.25,
        save=True,        # save images with bounding boxes
        save_txt=True     # save predictions as YOLO txt
    )

    print(f"Inference done! Check '{results[0].save_dir}' for results.")


image 1/100 C:\Users\yuval\PycharmProjects\GenAI_Project\content_new\kitti_dirty_subset_run2\training\image_2\000037.png: 224x640 (no detections), 14.9ms
image 2/100 C:\Users\yuval\PycharmProjects\GenAI_Project\content_new\kitti_dirty_subset_run2\training\image_2\000085.png: 224x640 (no detections), 11.5ms
image 3/100 C:\Users\yuval\PycharmProjects\GenAI_Project\content_new\kitti_dirty_subset_run2\training\image_2\000191.png: 224x640 (no detections), 22.0ms
image 4/100 C:\Users\yuval\PycharmProjects\GenAI_Project\content_new\kitti_dirty_subset_run2\training\image_2\000213.png: 224x640 (no detections), 11.6ms
image 5/100 C:\Users\yuval\PycharmProjects\GenAI_Project\content_new\kitti_dirty_subset_run2\training\image_2\000391.png: 224x640 (no detections), 70.9ms
image 6/100 C:\Users\yuval\PycharmProjects\GenAI_Project\content_new\kitti_dirty_subset_run2\training\image_2\000404.png: 224x640 (no detections), 64.6ms
image 7/100 C:\Users\yuval\PycharmProjects\GenAI_Project\content_new\kitti_

In [20]:
import glob

# Load predictions
pred_files = sorted(glob.glob("old_ver/runs/predict/exp/labels/*.txt"))
# Load ground truth
gt_files = sorted(glob.glob(os.path.join(labels_path, "*.txt")))

print(f"Number of predictions: {len(pred_files)}")
print(f"Number of ground truth files: {len(gt_files)}")


Number of predictions: 0
Number of ground truth files: 100


In [21]:
# Example: count images with zero predictions
no_pred_count = 0
for f in pred_files:
    with open(f) as file:
        if len(file.readlines()) == 0:
            no_pred_count += 1

print(f"Images with no detected objects: {no_pred_count} / {len(pred_files)}")


Images with no detected objects: 0 / 0


In [22]:
from PIL import Image

pred_images = sorted(glob.glob("old_ver/runs/predict_clean/exp/*.jpg"))  # predicted images with boxes

# Show 5 sample images
for img_path in pred_images[:5]:
    img = Image.open(img_path)
    plt.figure(figsize=(8,6))
    plt.imshow(img)
    plt.axis('off')
    plt.show()


In [23]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

gt_path = "old_ver/guy/kitti_dirty_subset_run2/training/label_2"
dirty_pred_path = "old_ver/runs/detect/predict/labels"
clean_pred_path = "old_ver/runs/detect/predict2/labels"


In [24]:
def count_boxes(label_file):
    if not os.path.exists(label_file):
        return 0
    with open(label_file) as f:
        return len(f.readlines())

def collect_stats(pred_path, gt_path):
    stats = []
    for gt_file in glob.glob(os.path.join(gt_path, "*.txt")):
        name = os.path.basename(gt_file)
        gt_boxes = count_boxes(gt_file)
        pred_boxes = count_boxes(os.path.join(pred_path, name))
        stats.append({
            "image": name.replace(".txt", ""),
            "gt_boxes": gt_boxes,
            "pred_boxes": pred_boxes,
            "missed": gt_boxes > 0 and pred_boxes == 0,
            "false_positive": gt_boxes == 0 and pred_boxes > 0
        })
    return pd.DataFrame(stats)


In [25]:
dirty_df = collect_stats(dirty_pred_path, gt_path)
clean_df = collect_stats(clean_pred_path, gt_path)

dirty_df.head(), clean_df.head()


(    image  gt_boxes  pred_boxes  missed  false_positive
 0  000037         3           5   False           False
 1  000085         6           7   False           False
 2  000191        11          10   False           False
 3  000213        14          16   False           False
 4  000391         6           8   False           False,
     image  gt_boxes  pred_boxes  missed  false_positive
 0  000037         3          10   False           False
 1  000085         6          14   False           False
 2  000191        11          19   False           False
 3  000213        14          33   False           False
 4  000391         6          17   False           False)

In [26]:
summary = pd.DataFrame({
    "Condition": ["Clean", "Dirty"],
    "Images": [len(clean_df), len(dirty_df)],
    "Missed detections": [
        clean_df["missed"].sum(),
        dirty_df["missed"].sum()
    ],
    "Avg predictions/image": [
        clean_df["pred_boxes"].mean(),
        dirty_df["pred_boxes"].mean()
    ]
})

summary


,Condition,Images,Missed detections,Avg predictions/image
0,Clean,100,0,12.98
1,Dirty,100,2,6.28


In [27]:
plt.bar(
    ["Clean", "Dirty"],
    [clean_df["missed"].sum(), dirty_df["missed"].sum()]
)
plt.ylabel("Number of missed detections")
plt.title("Missed detections: Clean vs Dirty")
plt.show()


<Figure size 640x480 with 1 Axes>

In [28]:
plt.boxplot(
    [clean_df["pred_boxes"], dirty_df["pred_boxes"]],
    labels=["Clean", "Dirty"]
)
plt.ylabel("Predicted boxes per image")
plt.title("Detection density comparison")
plt.show()


C:\Users\yuval\AppData\Local\Temp\ipykernel_29084\2570332022.py:1: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(


<Figure size 640x480 with 1 Axes>

In [29]:
dirty_failures = dirty_df[dirty_df["missed"] == True]
dirty_failures.head(10)


,image,gt_boxes,pred_boxes,missed,false_positive
91,006485,2,0,True,False
92,006517,6,0,True,False


In [30]:
from PIL import Image
import matplotlib.pyplot as plt
import os

fail_example = dirty_failures.iloc[0]["image"]

# Try all common image extensions
img_path = None
for ext in [".jpg", ".png", ".jpeg"]:
    candidate = os.path.join(
        "old_ver/guy/kitti_dirty_subset_run2/training/image_2",
        fail_example + ext
    )
    if os.path.exists(candidate):
        img_path = candidate
        break

print("Using image:", img_path)

assert img_path is not None, "Image file not found"

# Load image
img = Image.open(img_path).convert("RGB")

# Force redraw
plt.figure(figsize=(6, 4))
plt.imshow(img)
plt.axis("off")
plt.title("Missed detection due to contamination")
plt.show()


Using image: content_new/kitti_dirty_subset_run2/training/image_2\006485.png


<Figure size 600x400 with 1 Axes>